<a href="https://colab.research.google.com/github/petteriTeikari/minivess_mlops/blob/main/notebooks/train_custom_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use a custom model quickly

Reuse the data(loader) from the "ML pipe"

Import the libraries needed

In [28]:
import sys
import os

If you run this Notebook on Colab, you need to install the Virtual Environment with Poetry yourself (what I understood):

In [39]:
running_in_colab = 'google.colab' in str(get_ipython())
if running_in_colab:
    print('You are running this on COLAB so installing the environment here')
    # TODO! Colab by default uses Python 3.10 so force the 3.8.17 here?
    os.chdir("/content")
    !git clone https://github.com/petteriTeikari/minivess_mlops.git
    !pip install poetry
    os.chdir("/content/minivess_mlops")
    !poetry config virtualenvs.in-project true
    # Running "!poetry install --no-ansi" is needed or not?
    !poetry install
    #!poetry shell
    # https://stackoverflow.com/a/65440080/6412152
else:
    print('Assuming that you are runnign this from IDE,\n'
          'or some other environment where you have your Jupyter kernel created from Poetry files')

Import the other modules now available from Poetry environment and from the Github repo (see [in_Google_Colab.ipynb](https://github.com/elise-chin/poetry-and-colab/blob/main/Using_python_poetry_in_Google_Colab.ipynb))

In [40]:
!poetry show -v # /content/minivess_mlops/.venv

In [31]:
if running_in_colab:
  VENV_BASE = '/content/minivess_mlops/.venv'
  VENV_PATH = os.path.join(VENV_BASE, 'lib', 'python3.10', 'site-packages')
  #VENV_PATH = "/content/gdrive/MyDrive/test-poetry/.venv/lib/python3.8/site-packages"
  #!mkdir /content/venv
  LOCAL_VENV_PATH = '/content/venv_poetry' # local notebook
  try:
    os.symlink(VENV_PATH, LOCAL_VENV_PATH) # connect to directory in drive
  except Exception as e:
    print('symlink failed, e = {}'.format(e))
  sys.path.insert(0, LOCAL_VENV_PATH)

  sys.path.insert(0,'/content/minivess_mlops')

In [32]:
from loguru import logger

from src.run_training import parse_args_to_dict
from src.training.experiment import define_experiment_data
from src.utils.config_utils import import_config

Import helper subfunction(s)

In [33]:
def get_dataloaders(experim_dataloaders: dict):
    # Get the "validation" and "train" dataloaders from the dictionary
    fold_name = 'fold0'
    split_names = list(experim_dataloaders[fold_name].keys())
    fold_key = experim_dataloaders.get(fold_name)
    if fold_key is not None:
        try:
            train = experim_dataloaders[fold_name]['TRAIN']
            val = experim_dataloaders[fold_name]['VAL']['MINIVESS']
        except Exception as e:
            raise IOError('Could not get the dataloaders from the dictionary, error = {}'.format(e))
    else:
        raise IOError('Fold name = "{}" not found in the dataloaders dictionary'.format(fold_name))
    return train, val

Input arguments for the training (you can add all the input arguments supported by `run_training.py` here

In [34]:
input_args = ['-c', 'tutorials/train_demo']

# Fake these as coming from the command line to match the main code (run_training.py)
sys.argv = ['notebook_run']  # Jupyter has all the extra crap, so replace that with this
for sysargv in input_args:
    sys.argv.append(sysargv)
args = parse_args_to_dict()

Create the config with Hydra from the .yaml file(s)

In [35]:
from src.utils.config_utils import config_import_script, set_up_experiment_run

config = config_import_script(args['task_config_file'],
                              parent_dir_string = '.', #f'..{os.sep}..',
                              parent_dir_string_defaults = f'..{os.sep}..')
print(config)

In [36]:
if running_in_colab:
  os.makedirs('volumes', exist_ok=True)

  data_dir = os.path.join('volumes', 'minivess-dvc-cache')
  os.makedirs(data_dir, exist_ok=True)
  args['data_dir'] = data_dir

  arfifacts_dir = os.path.join('volumes', 'minivess-artifacts')
  os.makedirs(arfifacts_dir, exist_ok=True)
  args['output_dir'] = arfifacts_dir

In [37]:
exp_run = set_up_experiment_run(config=config,
                                args=args,
                                log_level='DEBUG')
print(exp_run)

Import the dataloaders (now the data augmentations are here as well as data transformations)

In [41]:
_, _, experim_dataloaders, exp_run = (
        define_experiment_data(config=config,
                               exp_run=exp_run))

# Get the "validation" and "train" dataloaders from the dictionary
train, val = get_dataloaders(experim_dataloaders)

TypeError: ignored

Now you are ready to train your new model that you just wanna quickly test without
wanting to have a battle with the config .YAML files

Add maybe some fastai demo with MLflow autologging:
[https://github.com/mlflow/mlflow/blob/master/examples/fastai/train.py](https://github.com/mlflow/mlflow/blob/master/examples/fastai/train.py)

In [ ]:
# Iterate the dataloaders for demo
no_of_epochs = 3
logger.info('Training for {} epochs'.format(no_of_epochs))
for epoch in range(no_of_epochs):

    logger.info('Epoch {}/{}'.format(epoch, no_of_epochs - 1))

    # Train
    logger.info('train with {} batches'.format(len(train)))
    for i, batch in enumerate(train):
        images, mask = batch['image'], batch['label']

    # Validation
    logger.info('validate with {} batches'.format(len(train)))
    for j, batch in enumerate(val):
        images, mask = batch['image'], batch['label']

logger.info('Training done!')